### Imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import matplotlib.colors as mcolors
from neuropy import plotting
from neuropy.utils import signal_process
from scipy.ndimage import gaussian_filter1d
from scipy import stats
import subjects
import pandas as pd

### Ripple detection method schematic

In [ ]:
sess = subjects.sd.ratSday3[0]
# plt.style.use('dark_background')
fig = plotting.Fig(grid=(3, 1), wspace=0.5)
maze = sess.paradigm["maze"].flatten()
rpls = sess.pbe.time_slice(maze[0], maze[1])
channels = sess.probegroup.get_connected_channels()

rpl_indx = [829]  # visually identified with random selection runs
print(rpl_indx)
cmap = "gist_rainbow"
for i, indx in enumerate(rpl_indx):
    ax = plt.subplot(fig.gs[0])
    rpl = rpls[indx].flatten()
    rpl_sig = sess.eegfile.get_signal(
        channels[8][9], t_start=rpl[0] - 0.20, t_stop=rpl[1] + 0.2
    )
    plotting.plot_signal_traces(rpl_sig, pad=1, ax=ax, lw=2, color="b")
    ax.set_yticks([])

### Example ripple trace

In [ ]:
sess = subjects.sd.ratSday3[0]
# plt.style.use('dark_background')
figure = plotting.Fig()
fig, gs = figure.draw(grid=(3, 1), wspace=0.5)
maze = sess.paradigm["maze"].flatten()
rpls = sess.pbe.time_slice(maze[0], maze[1])
channels = sess.probegroup.get_connected_channels()

rpl_indx = [829]  # visually identified with random selection runs
print(rpl_indx)
cmap = "gist_rainbow"
for i, indx in enumerate(rpl_indx):
    ax = plt.subplot(gs[0])
    rpl = rpls[indx].flatten()
    rpl_sig = sess.eegfile.get_signal(
        channels[8][9], t_start=rpl[0] - 0.20, t_stop=rpl[1] + 0.2
    )
    plotting.plot_signal_traces(rpl_sig, pad=1, ax=ax, lw=2, color="b")
    ax.set_yticks([])

# figure.savefig(subjects.figpath_sd/'example_ripple')

### Ripple traces multiple shanks

In [ ]:
import numpy as np
import subjects
import matplotlib.pyplot as plt
from neuropy.plotting import plot_signal_traces

sess = subjects.tn.ratKday3[0]
t1 = 349 * 60 + 56 + 0.9
# t1 = 350*60+0.2
t2 = t1 + 2
fig, ax = plt.subplots(5, 1)
cmaps = ["r", "#30f83a", "#f830ee", "#1eb1f6", "#f6ee1e"]
for i in range(5):
    channels = sess.recinfo.channel_groups[i + 1].astype("int")
    signal = sess.eegfile.get_signal(channels, t1, t2)
    plot_signal_traces(signal, ax=ax[i], pad=0.2, color=cmaps[i], lw=0.5)
    ax[i].axis("off")
    # ax[i].set_facecolor('k')

fig.savefig(subjects.figpath_sd / "rpl_traces_multi_shanks.pdf")

### Ripple autocorrelation

In [ ]:
autocorr = subjects.GroupData().ripple_autocorr

fig = plotting.Fig(grid=(7, 3), fontsize=7)
ax = fig.subplot(fig.gs[0])
sns.lineplot(
    data=autocorr,
    x="lag_time",
    y="auto_corr",
    hue="grp",
    ci=None,
    palette=subjects.colors_sd(),
    hue_order=["NSD", "SD"],
)
ax.legend("", frameon=False)
fig.savefig(subjects.figpath_sd / "ripple_autocorr")

### Example ripples and replay raster plot below it

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import subjects
from neuropy import plotting
from neuropy.analyses import Pf1D

sess = subjects.nsd.ratVday3[0]

figure = plotting.Fig()
fig, gs = figure.draw(grid=(3, 5), wspace=0.5)
maze = sess.paradigm["maze"].flatten()
rpls = sess.pbe.time_slice(maze[0], maze[1])
channels = sess.probegroup.get_connected_channels()
pyr_neurons = sess.neurons.get_neuron_type("pyr")
pos = sess.lin_maze
# pos.t_start = pos.t_start
run = sess.run["forward"]
pf = Pf1D(
    neurons=pyr_neurons,
    position=pos,
    speed_thresh=10,
    sigma=4,
    grid_bin=3,
    frate_thresh=1,
    epochs=sess.run,
)
pf_neurons = pyr_neurons.get_by_id(pf.neuron_ids)

tc = pf.tuning_curves
sort_ind = np.argsort(np.argmax(tc, axis=1))
pf_neurons = pf_neurons[sort_ind]

# rpl_indx = np.random.choice(len(rpls),4,replace=False)
rpl_indx = [356, 712, 426, 829]  # visually identified with random selection runs
print(rpl_indx)
cmap = "gist_rainbow"
for i, indx in enumerate(rpl_indx):
    ax = plt.subplot(gs[0, i + 1])
    rpl = rpls[indx].flatten()
    rpl_sig = sess.eegfile.get_signal(
        channels[8].astype(int), t_start=rpl[0] - 0.10, t_stop=rpl[1] + 0.1
    )
    plotting.plot_signal_traces(rpl_sig, pad=1, ax=ax, lw=0.5, color="k")
    if i > 0:
        ax.set_yticks([])

    neurons = pf_neurons.time_slice(rpl[0], rpl[1])
    ax_raster = plt.subplot(gs[1, i + 1])
    ax_raster = plotting.plot_raster(neurons, ax=ax_raster, color=cmap, markersize=4)

    ax_raster.axis("off")

ax_pf = plt.subplot(gs[1, 0])
plotting.plot_ratemap(pf, ax=ax_pf, normalize_tuning_curve=True, cmap=cmap)
ax_pf.set_yticks([])
# figure.savefig(subjects.figpath_sd/'example_ripples_')

### Schematic of sharp wave amplitude method
- for use in sd_figure2

In [ ]:
from neuropy.utils.signal_process import filter_sig

sess = subjects.nsd.ratUday2[0]
sw_amp = sess.ripple.to_dataframe()["sharp_wave_amp"]
# indx = np.random.choice(np.where(sw_amp>3)[0])
indx = 17766
period = sess.ripple[indx].flatten()
period = [period[0] - 0.1, period[1] + 0.1]
channels = sess.recinfo.channel_groups[6].astype("int")
signal = sess.eegfile.get_signal(channels, *period)
filt_signal = filter_sig.bandpass(signal, lf=2, hf=30)

# _,ax = plt.subplots()
fig = plotting.Fig(grid=(10, 19))
# fig = plotting.Fig(grid=(5,7))
ax = fig.subplot(fig.gs[0, :2])
plotting.plot_signal_traces(signal, ax=ax, pad=0.3, color="#949494", lw=0.3)
# plotting.plot_signal_traces(filt_signal,ax=ax,color='#5d22a0',pad=0.4,lw=0.5)
ax.axis("off")

fig.savefig(subjects.figpath_sd / "sw_amp_example")

### Spectrogram in the ripple band showing ripple frquency and power changes NSD vs SD

In [ ]:
from neuropy.utils.signal_process import FourierSg, filter_sig
from hfuncs import whiten_signal

sessions = subjects.nsd.ratVday1 + subjects.sd.ratVday2

ripple_spects = []
for sub, sess in enumerate(sessions):
    post = sess.paradigm["post"].flatten()
    period = [post[0], post[0] + 7.5 * 3600]
    rpl_df = sess.ripple.time_slice(*period).to_dataframe()
    signal = sess.eegfile.get_signal(82).time_slice(None, *period)
    signal = filter_sig.bandpass(signal, lf=125, hf=250)
    # signal = whiten_signal(signal)
    spect = FourierSg(signal, window=4, overlap=2).freq_slice(125, 250)
    ripple_spects.append(
        dict(
            sxx=spect,
            time=rpl_df.start.values,
            freq=rpl_df.peak_frequency_bp,
            states=sess.brainstates.time_slice(*period),
        )
    )

In [ ]:
from hfuncs import merge_quiet_wake

fig = plotting.Fig(grid=(7, 12), constrained_layout=False)
subfig = fig.add_subfigure(fig.gs[0, :8])
axs = subfig.subplots(
    2, 2, gridspec_kw=dict(height_ratios=[1, 4]), sharex=True, sharey="row"
)
# _, axs = plt.subplots(9,12)
example_ripple_timepoints = [
    [1.66, 3.65, 5.39],
    [1.93, 4.14, 5.51],
]  # selected from example rippples cells

for i, spect in enumerate(ripple_spects):
    sxx = spect["sxx"]
    post_t = np.arange(sxx.time[0], sxx.time[-1] + 5, 2.5 * 3600)
    zt_time = (post_t - sxx.time[0]) / 3600

    ax = axs[0, i]
    slp_states = merge_quiet_wake(spect["states"])
    plotting.plot_epochs(
        epochs=slp_states, ax=ax, labels_order=["NREM", "REM", "WK"], colors="k"
    )
    ax.set_xticks(post_t, zt_time)
    ax.set_axis_off()

    ax = axs[1, i]
    im = ax.imshow(
        stats.zscore(sxx.traces, axis=None),
        aspect="auto",
        # vmin=-0.,
        vmax=1,
        cmap="plasma",
        # norm=mcolors.LogNorm(0.0000001,0.07),
        origin="lower",
        extent=[sxx.time[0], sxx.time[-1], sxx.freqs[0], sxx.freqs[-1]],
    )

    rpl_t = np.array(example_ripple_timepoints[i]) * 3600 + sxx.time[0]
    ax.plot(rpl_t, [132, 132, 132], "v", color="w", markersize=3)

    mov_window = (1 / 20) * np.ones(20)
    ax.plot(
        spect["time"], np.convolve(spect["freq"], mov_window, mode="same"), "k", lw=0.2
    )
    ax.set_ylim(125, 250)
    ax.set_xticks(post_t, zt_time)

ax = ax.inset_axes([1, 0, 0.1, 1])
plt.colorbar(im, ax=ax)
ax.set_axis_off()
fig.savefig(subjects.figpath_sd / "ripple_spect")

### Example ripples from NSD, SD showing varying sharp wave amplitude

In [ ]:
from neuropy.core import Signal

sessions = subjects.nsd.ratVday1 + subjects.sd.ratVday2

rng = np.random.default_rng()

# _,axs = plt.subplots(2,3,sharey=True,sharex=True)
fig = plotting.Fig(grid=(5, 4))
subfig = fig.add_subfigure(fig.gs[:3, :3])
axs = subfig.subplots(2, 3, sharey=True, sharex=True)

ripple_spects = []
for sub, sess in enumerate(sessions):
    post = sess.paradigm["post"].flatten()
    # channels = np.concatenate([(f:=sess.recinfo.channel_groups)[5],f[6],f[7]])
    channels = sess.recinfo.channel_groups[5]
    periods = [
        [post[0], post[0] + 2 * 3600],
        [post[0] + 3 * 3600, post[0] + 5 * 3600],
        [post[0] + 5 * 3600, post[0] + 7 * 3600],
    ]

    if sess.tag == "NSD":
        rpl_indx = [3510, 1087, 798]
    if sess.tag == "SD":
        rpl_indx = [3737, 2452, 945]

    # traces = []
    for i, period in enumerate(periods):
        rpls = sess.ripple.time_slice(*period)

        if rpl_indx[i] is None:
            df = rpls.to_dataframe()
            spw_amp = df.sharp_wave_amp.values
            perc_amp = np.percentile(spw_amp, 95)
            high_indx = np.where(spw_amp >= perc_amp)[0]
            indx = rng.choice(high_indx, 1)
        else:
            indx = rpl_indx[i]

        rpl_t = rpls[indx].flatten()
        rpl_signal = sess.eegfile.get_signal(channels, rpl_t[0], rpl_t[1])
        rpl_signal.t_start = 0
        # traces.append(sess.eegfile.get_signal(channels,rpl_t[0],rpl_t[1]).traces)

        # rpl_signal = Signal(np.hstack(traces),sampling_rate=1250)
        ax = axs[sub, i]
        plotting.plot_signal_traces(rpl_signal, ax=ax, lw=0.4, pad=0.2)
        ax.set_title(f"{indx},{((rpl_t[0]-post[0])/3600).round(2)}")
        ax.set_axis_off()

fig.savefig(subjects.figpath_sd / "spwr_examples")

### Schematic Ripple associated features
- for use in thesis defense

In [ ]:
from neuropy.utils.signal_process import filter_sig, WaveletSg

sess = subjects.nsd.ratUday2[0]
sw_amp = sess.ripple.to_dataframe()["sharp_wave_amp"]
# indx = np.random.choice(np.where(sw_amp>3)[0])

indx = 17766
period = sess.ripple[indx].flatten()
period = [period[0] - 0.1, period[1] + 0.1]
channels = sess.recinfo.channel_groups[6].astype("int")
signal = sess.eegfile.get_signal(channels, *period)
filt_signal = filter_sig.bandpass(signal, lf=2, hf=30)

fig = plotting.Fig(grid=(17, 5), fontsize=10, axis_lw=2)

ax = fig.subplot(fig.gs[0:5, :2])
plotting.plot_signal_traces(signal, ax=ax, pad=0.3, color="k", lw=1)
ax.axis("off")

indx = 11015
selected_rippls = sess.ripple[indx : indx + 3].flatten()
period = [selected_rippls[0] - 0.4, selected_rippls[-1] - 0.5]
# period = [369*60+47,369*60+47+1.2]
# channels = sess.recinfo.channel_groups[6].astype('int')
signal = sess.eegfile.get_signal(99, *period)
spect = WaveletSg(signal, freqs=np.arange(90, 300, 2), ncycles=10)
filt_signal = filter_sig.bandpass(signal, lf=125, hf=250)


ax = fig.subplot(fig.gs[0, 2:])
ax.plot(signal.time, signal.traces[0], color="k")
ax.axis("off")

ax = fig.subplot(fig.gs[1, 2:], sharex=ax)
ax.plot(signal.time, filt_signal.traces[0], color="k")
ax.axis("off")

ax = fig.subplot(fig.gs[2:4, 2:], sharex=ax)
plotting.plot_signal_heatmap(spect, ax=ax, cmap="jet", rasterized=True)
# ax.imshow(spect.traces,extent=[0,10,100,300],aspect='auto',cmap='jet',origin='lower')
ax.set_ylabel("Frequency (Hz)")
# ax.axis('off')


fig.savefig(subjects.figpath_sd / "swr_parameters")